In [ ]:
import numpy as np

def mic_displacement_ortho(r, r0, L):
    """Minimum-image displacement (orthorhombic boxes).
    r, r0: (N,3), L: (3,)
    """
    dr = r - r0
    L = np.asarray(L)[None, :]
    dr -= L * np.round(dr / L)
    return dr


In [2]:
def gr_single_frame(positions, box_lengths, dr=0.01, r_max=None, sample=None):
    """
    Compute radial distribution function g(r) for one frame.

    positions : (N,3) float64
    box_lengths : (3,) [Lx, Ly, Lz] orthorhombic
    dr : bin width
    r_max : maximum r; default Lmin/2
    sample : if not None, subsample 'sample' particles as references to reduce O(N^2) cost

    Returns:
        r_centers : (B,)
        g_r       : (B,)
    """
    r = np.asarray(positions, dtype=np.float64)
    L = np.asarray(box_lengths, dtype=np.float64)
    V = L.prod()
    N = r.shape[0]
    rho = N / V

    if r_max is None:
        r_max = 0.5 * np.min(L)  # g(r) well-defined only up to half the smallest box length

    # choose reference indices for pair distances
    if sample is None or sample >= N:
        ref_idx = np.arange(N, dtype=int)
    else:
        rng = np.random.default_rng()
        ref_idx = rng.choice(N, size=int(sample), replace=False)

    # histogram bins
    edges = np.arange(0.0, r_max + dr, dr)
    counts = np.zeros(edges.size - 1, dtype=np.float64)

    # chunk to manage memory: compute distances from refs to all
    chunk = max(1, 10_000 // max(1, N))  # heuristic small chunk
    for i0 in range(0, len(ref_idx), chunk):
        ii = ref_idx[i0:i0+chunk]
        # (m,1,3) - (1,N,3) -> (m,N,3)
        disp = mic_displacement_ortho(r[None, :, :], r[ii, None, :], L)   # careful: broadcast ref as r0
        # distances to all particles (including self -> zeros; we will drop them)
        dist = np.linalg.norm(disp, axis=2).ravel()

        # drop self-distances (zeros)
        dist = dist[dist > 1e-12]
        # grab only within r_max
        dist = dist[dist < r_max]

        # histogram
        c, _ = np.histogram(dist, bins=edges)
        counts += c

    # normalization: average number density in spherical shells
    # number of reference particles used:
    n_ref = len(ref_idx)
    shell_vol = (4.0/3.0) * np.pi * (edges[1:]**3 - edges[:-1]**3)  # dV = 4/3 π (r_{i+1}^3 - r_i^3)
    ideal_counts = n_ref * rho * shell_vol

    # g(r) = observed / ideal
    g_r = counts / ideal_counts
    r_centers = 0.5 * (edges[1:] + edges[:-1])
    return r_centers, g_r


In [3]:
import gsd.hoomd

def gr_from_gsd(gsd_path, dr=0.01, r_max=None, stride=1, start=0, stop=None, sample=None):
    """
    Compute g(r) averaged over frames in a GSD (orthorhombic box).
    Returns:
        r  : (B,)
        g  : (B,)
    """
    with gsd.hoomd.open(gsd_path, 'r') as traj:
        if stop is None:
            stop = len(traj)

        # prepare from first frame
        f0 = traj[start]
        L = np.array(f0.configuration.box[:3], dtype=np.float64)
        if r_max is None:
            r_max = 0.5 * np.min(L)

        # accumulate counts to average
        edges = np.arange(0.0, r_max + dr, dr)
        accum_counts = np.zeros(edges.size - 1, dtype=np.float64)
        accum_refs = 0
        rho = None  # density should be constant; verify first frame

        for t in range(start, stop, stride):
            fr = traj[t]
            r = fr.particles.position.astype(np.float64)
            box = fr.configuration.box
            if not np.allclose(box[3:], 0.0):  # xy,xz,yz nonzero → triclinic
                raise RuntimeError("Triclinic box detected. Use freud for MIC in triclinic boxes.")

            L = np.array(box[:3], dtype=np.float64)
            V = L.prod()
            N = r.shape[0]
            rho_t = N / V
            if rho is None:
                rho = rho_t
            # compute per-frame histogram, reusing the per-frame function but exposing internals
            # (avoid repeated bin creation)
            # We duplicate minimal logic here for performance:
            if sample is None or sample >= N:
                ref_idx = np.arange(N, dtype=int)
            else:
                rng = np.random.default_rng()
                ref_idx = rng.choice(N, size=int(sample), replace=False)

            counts = np.zeros_like(accum_counts)
            chunk = max(1, 10_000 // max(1, N))
            for i0 in range(0, len(ref_idx), chunk):
                ii = ref_idx[i0:i0+chunk]
                disp = mic_displacement_ortho(r[None, :, :], r[ii, None, :], L)
                dist = np.linalg.norm(disp, axis=2).ravel()
                dist = dist[dist > 1e-12]
                dist = dist[dist < r_max]
                c, _ = np.histogram(dist, bins=edges)
                counts += c

            accum_counts += counts
            accum_refs  += len(ref_idx)

        # finalize average
        shell_vol = (4.0/3.0) * np.pi * (edges[1:]**3 - edges[:-1]**3)
        ideal_counts = accum_refs * rho * shell_vol
        g_r = accum_counts / ideal_counts
        r_centers = 0.5 * (edges[1:] + edges[:-1])
        return r_centers, g_r


In [10]:
import numpy as np
import gsd.hoomd
from pathlib import Path

# ---------- MIC helper (orthorhombic boxes) ----------
def _mic(dr, L):
    return dr - L * np.round(dr / L)

# ---------- per-file histogram from the last frame ----------
def _gr_counts_lastframe(gsd_path, edges, r_max, sample=None):
    with gsd.hoomd.open(gsd_path, 'r') as traj:
        fr = traj[-1]  # last snapshot
        r   = fr.particles.position.astype(np.float64)
        box = fr.configuration.box  # [Lx,Ly,Lz,xy,xz,yz]
        if not np.allclose(box[3:], 0.0):
            raise RuntimeError("Triclinic box detected; use freud for triclinic MIC.")
        L = np.array(box[:3], dtype=np.float64)
        V = float(np.prod(L))
        N = r.shape[0]
        rho = N / V

    # choose reference set (optional subsampling)
    if (sample is None) or (sample >= N):
        ref_idx = np.arange(N, dtype=int)
    else:
        rng = np.random.default_rng()
        ref_idx = rng.choice(N, size=int(sample), replace=False)

    counts = np.zeros(edges.size - 1, dtype=np.float64)

    # chunk over references for memory safety
    chunk = max(1, 2000)
    for i0 in range(0, len(ref_idx), chunk):
        ii = ref_idx[i0:i0+chunk]
        dr = r[None, :, :] - r[ii, None, :]
        dr = _mic(dr, L)
        dist = np.linalg.norm(dr, axis=2).ravel()
        dist = dist[(dist > 1e-12) & (dist < r_max)]
        c, _ = np.histogram(dist, bins=edges)
        counts += c

    return counts, len(ref_idx), rho

# ---------- average g(r) over many files (last frame of each) ----------
def gr_avg_lastframes(files, dr=0.01, r_max=None, sample=None):
    files = [str(f) for f in files]

    # decide common r_max if not provided
    if r_max is None:
        rmaxs = []
        for p in files:
            with gsd.hoomd.open(p, 'r') as traj:
                box = traj[-1].configuration.box
                if not np.allclose(box[3:], 0.0):
                    raise RuntimeError("Triclinic box detected; use freud for triclinic MIC.")
                L = np.array(box[:3], dtype=np.float64)
                rmaxs.append(0.5 * float(np.min(L)))
        r_max = float(np.min(rmaxs))

    edges = np.arange(0.0, r_max + dr, dr)
    shell_vol = (4.0/3.0) * np.pi * (edges[1:]**3 - edges[:-1]**3)

    total_counts = np.zeros(edges.size - 1, dtype=np.float64)
    total_ideal  = np.zeros_like(total_counts)

    for p in files:
        counts, n_ref, rho = _gr_counts_lastframe(p, edges, r_max, sample=sample)
        total_counts += counts
        total_ideal  += n_ref * rho * shell_vol

    g_avg = total_counts / total_ideal
    r_centers = 0.5 * (edges[1:] + edges[:-1])
    return r_centers, g_avg

# ---------- convenience: compute & SAVE with filename metadata ----------
def save_gr_avg_with_meta(read_dir, out_dir, *, T, rho, ep, wait, dr=0.01, r_max=None, sample=None, prec=6):
    """
    files : list of .gsd paths
    CSV   : r,g(r) only; filename encodes T,rho,ep,wait
    """
    rho_tag  = f"{rho:.{6}f}"
    T_tag    = f"{T:.{8}f}"
    ep_tag = f"{ep:.{0}f}"
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    read_dir = Path(read_dir)
    files = []
    for i,w in enumerate(wait):
        files.append(read_dir / f"traj_T{T_tag}_rho{rho_tag}_ep{ep_tag}_wait{str(int(w))}.gsd")
    r, g = gr_avg_lastframes(files, dr=dr, r_max=r_max, sample=sample)



    csv_path = out_dir / f"gr_avg_T{T_tag}_rho{rho_tag}_ep{ep_tag}.csv"
    np.savetxt(csv_path, np.column_stack([r, g]), delimiter=",", header="r,g(r)", comments="")
    return 0


In [13]:
wait_lists = np.linspace(100000, 1000000, 10)
save_gr_avg_with_meta(
    "/home/chengling/Research/Project/vitrimer/data/test/RevEp/rho0.923333", out_dir="/home/chengling/Research/Project/vitrimer/data/test/RevEp/rho0.923333/static",
    T=0.001, rho=0.923333, ep=1000.0, wait=wait_lists,
    dr=0.02, r_max=6, sample=None  # sample=None for exact references
)


0